In [1]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [2]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)

In [3]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #number of genres we are looking at
        for i in range(numElems):
            genre_str = (json_genres[i]['name'])
            if genre_str in genre_map.keys():
                ret[genre_dict[genre_map[genre_str]]] = 1
        return ret
    except Exception as excep:
        print('Exception' + str(excep))
        return ''

In [4]:
genre_dict = {}
genre_dict['Action-Adventure'] = 0
genre_dict['Romance'] = 1
genre_dict['Horror-Thriller'] = 2
genre_dict['Comedy'] = 3
genre_dict['Science Fiction'] = 4
#genre_dict['Drama'] = 5
genre_dict

{'Action-Adventure': 0,
 'Romance': 1,
 'Horror-Thriller': 2,
 'Comedy': 3,
 'Science Fiction': 4}

In [5]:
genre_map = {}
genre_map['Adventure'] = 'Action-Adventure'
genre_map['Romance'] = 'Romance'
genre_map['Horror'] = 'Horror-Thriller'
genre_map['Thriller'] = 'Horror-Thriller'
genre_map['Comedy'] = 'Comedy'
#genre_map['War'] = 'Action-Adventure'#not sure about this
genre_map['Action'] = 'Action-Adventure'
genre_map['Science Fiction'] = 'Science Fiction'
#genre_map['Drama'] = 'Drama'

In [6]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    return ret

In [7]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [8]:
#put to lower case, remove punctation
def cleanText(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text)
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [9]:
all_data = all_data[all_data.genres_vect.map(sum) > 0]

In [10]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_vect', 'overview']]

In [11]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [12]:
#get word embeddings
x = train['cleanOverview'].values.tolist()
y = train['genres_vect']

In [13]:
x_test = test['cleanOverview'].values.tolist()
y_test = test['genres_vect']

In [14]:
y_train = y.tolist()
y_train = np.array(y_train)

In [15]:
y_test = y_test.tolist()
y_test = np.array(y_test)

In [16]:
tok = [word_tokenize(sent) for sent in x]

In [17]:
word_vec_len = 32
model = Word2Vec(tok, min_count = 2, size=word_vec_len)

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words_kept = 100000 #using 100000 most popular words, use throughout

tokenizer = Tokenizer(num_words_kept)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

max_seq_len = 150

x_train_seq = pad_sequences(sequences, maxlen=max_seq_len)

Using TensorFlow backend.


In [19]:
test_sequences = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(test_sequences, maxlen=max_seq_len)

In [20]:
embeddings_index = {}
for w in model.wv.vocab.keys():
    embeddings_index[w] = model.wv[w]


embedding_matrix = np.zeros((num_words_kept, word_vec_len))
for word, i in tokenizer.word_index.items():
    if i >= num_words_kept:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def get_per_label_metrics(real_labels_matrix, predictions_labels_matrix):
    for genre in genre_dict.keys():
        index = genre_dict[genre]
        real_labels_vect = real_labels_matrix[:, index]
        prediction_vect = predictions_labels_matrix[:,index]
        print("Accuruacy for " + genre + ": " + str(accuracy_score(real_labels_vect, prediction_vect)))
        print("Precision for " + genre + ": " + str(precision_score(real_labels_vect, prediction_vect)))
        print("Recall for " + genre + ": " + str(recall_score(real_labels_vect, prediction_vect)))
        print()

In [22]:
#size of intersection of predicted and actual labels divided by size of their union for each datapoint tested on
#sum those and then divide by number of datapoints
#vectorized for speed
def multi_label_accuracy(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    union = real_labels_matrix | predictions_labels_matrix
    #sum(array.T) gets number of 1s in row
    row_wise_accuracy = sum(intersection.T) / sum(union.T)
    return sum(row_wise_accuracy) / real_labels_matrix.shape[0]

#size of intersection of predicted and actual labels divided by size of predicted set for each datapoint tested on
#sum those and divide by number of datapoints
#if no predicted labels, don't count that row towards the precision as that would be undefined
def multi_label_precision(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    precision_sum = 0
    num_rows = 0
    for row in range(intersection.shape[0]):
        if sum(predictions_labels_matrix[row]) > 0: #if there is at least one prediction for this row
            num_rows += 1
            precision_sum += sum(intersection[row]) / sum(predictions_labels_matrix[row])
    if num_rows == 0:
        return 0#no labels predicted at all will give us 0 precision as precision makes no sense here
    return precision_sum / num_rows

#size of intersection of predicted and actual labels divided by size of real label set for each datapoint tested on
#sum those and divide by number of datapoints
#all datapoints should have at least 1 real label in this data set
#vectorized for speed
def multi_label_recall(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    #sum(array.T) gets number of 1s in row
    row_wise_recall = sum(intersection.T) / sum(real_labels_matrix.T)
    return sum(row_wise_recall) / real_labels_matrix.shape[0]

#lower is better
def hamming_loss(real_labels_matrix, predictions_labels_matrix):
    return (np.logical_xor(real_labels_matrix, predictions_labels_matrix)).sum()/(real_labels_matrix.shape[0] * real_labels_matrix.shape[1])

import keras.backend as K

#metric for keras for early stopping
#takes in raw labels from kerass (not yet converted to 0 and 1s)
#NOT the same as accuracy, this is total labels correctly identified divided by union of total labels
#this weights rows with more labels higher, where accruacy does not, but this is still a good metric for early stopping
def raw_multi_label_accuracy(y_true, y_pred):
    positives = K.greater_equal(y_pred, 0.5)
    positives = K.cast(positives, K.floatx())
    new_y_pred = positives #+ ((1-positives)*y_pred)
    intersection = y_true * new_y_pred
    union = 1 -((1-y_true)*(1-new_y_pred))
    accuracy = K.sum(intersection) / K.sum(union)
    return accuracy
    

In [23]:
from keras.callbacks import EarlyStopping
#for early stopping only after certain number of epochs. wait until delay epochs until early stopping
class DelayedEarlyStopping(EarlyStopping):
    def __init__(self, monitor, min_delta=0, patience=0, verbose=0, mode='auto', delay = 100):
        super(DelayedEarlyStopping, self).__init__()
        self.delay = delay

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.delay:
            super().on_epoch_end(epoch, logs)

In [24]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

model_cnn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
#e = Embedding(num_words_kept, word_vec_len, input_length=max_seq_len, trainable=True)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(50, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(len(genre_dict), activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
model_cnn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=200)], epochs=1000, batch_size=100, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 2s - loss: 2.1209 - raw_multi_label_accuracy: 0.1890 - val_loss: 1.8021 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 1s - loss: 1.6278 - raw_multi_label_accuracy: 0.1493 - val_loss: 1.4011 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 1s - loss: 1.2844 - raw_multi_label_accuracy: 0.1262 - val_loss: 1.1226 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 1s - loss: 1.0464 - raw_multi_label_accuracy: 0.1225 - val_loss: 0.9320 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 1s - loss: 0.8864 - raw_multi_label_accuracy: 0.0988 - val_loss: 0.8100 - val_raw_multi_label_accura

Epoch 55/1000
 - 1s - loss: 0.2938 - raw_multi_label_accuracy: 0.6610 - val_loss: 0.8267 - val_raw_multi_label_accuracy: 0.2780
Epoch 56/1000
 - 2s - loss: 0.2936 - raw_multi_label_accuracy: 0.6543 - val_loss: 0.7799 - val_raw_multi_label_accuracy: 0.2807
Epoch 57/1000
 - 1s - loss: 0.3000 - raw_multi_label_accuracy: 0.6661 - val_loss: 0.8073 - val_raw_multi_label_accuracy: 0.2915
Epoch 58/1000
 - 2s - loss: 0.2900 - raw_multi_label_accuracy: 0.6696 - val_loss: 0.7985 - val_raw_multi_label_accuracy: 0.2968
Epoch 59/1000
 - 1s - loss: 0.2850 - raw_multi_label_accuracy: 0.6709 - val_loss: 0.8708 - val_raw_multi_label_accuracy: 0.2762
Epoch 60/1000
 - 1s - loss: 0.2848 - raw_multi_label_accuracy: 0.6669 - val_loss: 0.8438 - val_raw_multi_label_accuracy: 0.2836
Epoch 61/1000
 - 1s - loss: 0.2797 - raw_multi_label_accuracy: 0.6912 - val_loss: 0.8351 - val_raw_multi_label_accuracy: 0.2808
Epoch 62/1000
 - 2s - loss: 0.2731 - raw_multi_label_accuracy: 0.7055 - val_loss: 0.8769 - val_raw_multi

Epoch 119/1000
 - 1s - loss: 0.2038 - raw_multi_label_accuracy: 0.7824 - val_loss: 1.2372 - val_raw_multi_label_accuracy: 0.2906
Epoch 120/1000
 - 1s - loss: 0.2006 - raw_multi_label_accuracy: 0.7781 - val_loss: 1.2398 - val_raw_multi_label_accuracy: 0.3001
Epoch 121/1000
 - 1s - loss: 0.1972 - raw_multi_label_accuracy: 0.7805 - val_loss: 1.2458 - val_raw_multi_label_accuracy: 0.3080
Epoch 122/1000
 - 1s - loss: 0.1996 - raw_multi_label_accuracy: 0.7786 - val_loss: 1.2040 - val_raw_multi_label_accuracy: 0.3081
Epoch 123/1000
 - 1s - loss: 0.1935 - raw_multi_label_accuracy: 0.7861 - val_loss: 1.2626 - val_raw_multi_label_accuracy: 0.3033
Epoch 124/1000
 - 1s - loss: 0.1926 - raw_multi_label_accuracy: 0.7815 - val_loss: 1.2317 - val_raw_multi_label_accuracy: 0.3028
Epoch 125/1000
 - 1s - loss: 0.1919 - raw_multi_label_accuracy: 0.7923 - val_loss: 1.2378 - val_raw_multi_label_accuracy: 0.3000
Epoch 126/1000
 - 1s - loss: 0.1949 - raw_multi_label_accuracy: 0.7858 - val_loss: 1.2409 - val_r

Epoch 183/1000
 - 1s - loss: 0.1561 - raw_multi_label_accuracy: 0.8613 - val_loss: 1.6218 - val_raw_multi_label_accuracy: 0.2828
Epoch 184/1000
 - 1s - loss: 0.1558 - raw_multi_label_accuracy: 0.8604 - val_loss: 1.6027 - val_raw_multi_label_accuracy: 0.2910
Epoch 185/1000
 - 1s - loss: 0.1540 - raw_multi_label_accuracy: 0.8624 - val_loss: 1.6237 - val_raw_multi_label_accuracy: 0.2922
Epoch 186/1000
 - 1s - loss: 0.1534 - raw_multi_label_accuracy: 0.8616 - val_loss: 1.6522 - val_raw_multi_label_accuracy: 0.2805
Epoch 187/1000
 - 1s - loss: 0.1553 - raw_multi_label_accuracy: 0.8629 - val_loss: 1.6223 - val_raw_multi_label_accuracy: 0.2881
Epoch 188/1000
 - 1s - loss: 0.1544 - raw_multi_label_accuracy: 0.8566 - val_loss: 1.6754 - val_raw_multi_label_accuracy: 0.2812
Epoch 189/1000
 - 1s - loss: 0.1522 - raw_multi_label_accuracy: 0.8627 - val_loss: 1.5769 - val_raw_multi_label_accuracy: 0.2964
Epoch 190/1000
 - 1s - loss: 0.1490 - raw_multi_label_accuracy: 0.8633 - val_loss: 1.5686 - val_r

In [25]:
def nn_output_to_predictions(res):
    label_predictions = []
    for i in range(res.shape[0]):
        pred = [0]*len(genre_dict)
        for j in range(res.shape[1]):
            if res[i][j] >= .5:
                pred[j] = 1
        label_predictions.append(pred)
    return np.array(label_predictions)

In [26]:
predictions = nn_output_to_predictions(model_cnn.predict(x_test_seq))

In [27]:
y_test[:,0].sum()

187

In [28]:
predictions[:,0].sum()

219

In [29]:
predictions[0]

array([1, 0, 0, 1, 0])

In [30]:
multi_label_accuracy(y_test, predictions)

0.3480676328502416

In [31]:
multi_label_precision(y_test, predictions)

0.5141509433962265

In [32]:
multi_label_recall(y_test, predictions)

0.43046928916494137

In [33]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 68.86128364389235


In [34]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.6356107660455487
Precision for Action-Adventure: 0.5251141552511416
Recall for Action-Adventure: 0.6149732620320856

Accuruacy for Romance: 0.7349896480331263
Precision for Romance: 0.3225806451612903
Recall for Romance: 0.18867924528301888

Accuruacy for Horror-Thriller: 0.6004140786749482
Precision for Horror-Thriller: 0.49645390070921985
Recall for Horror-Thriller: 0.3645833333333333

Accuruacy for Comedy: 0.6149068322981367
Precision for Comedy: 0.572972972972973
Recall for Comedy: 0.49765258215962443

Accuruacy for Science Fiction: 0.8571428571428571
Precision for Science Fiction: 0.15789473684210525
Recall for Science Fiction: 0.05357142857142857



CNN but with multiple filter sizes so we don't just filter on group of words at a time

In [35]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

model_input = Input(shape=(max_seq_len,), dtype='int32')

e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)(model_input)
two_word_filter = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(e)
two_word_filter = GlobalMaxPooling1D()(two_word_filter)
three_word_filter = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(e)
three_word_filter = GlobalMaxPooling1D()(three_word_filter)
four_word_filter = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(e)
four_word_filter = GlobalMaxPooling1D()(four_word_filter)
merged = concatenate([two_word_filter, three_word_filter, four_word_filter], axis=1)

merged = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(merged)
merged = Dropout(0.5)(merged)
merged = Dense(len(genre_dict))(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[model_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])

In [36]:
model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 4s - loss: 2.8855 - raw_multi_label_accuracy: 0.0847 - val_loss: 2.2943 - val_raw_multi_label_accuracy: 0.0294
Epoch 2/1000
 - 3s - loss: 1.9296 - raw_multi_label_accuracy: 0.0570 - val_loss: 1.5420 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 3s - loss: 1.3252 - raw_multi_label_accuracy: 0.0305 - val_loss: 1.0982 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 3s - loss: 0.9787 - raw_multi_label_accuracy: 0.0185 - val_loss: 0.8555 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 3s - loss: 0.7902 - raw_multi_label_accuracy: 0.0320 - val_loss: 0.7204 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 3s - loss: 0.6902 - raw_multi_label_accuracy: 0.0349 - val_loss: 0.6539 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 7/1000
 - 3s - loss: 0.6369 - raw_multi_label_accuracy: 0.0398 - val_loss: 0.6233 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 8/1000
 - 3s - loss: 0.5909 - raw_

In [37]:
predictions = nn_output_to_predictions(model.predict(x_test_seq))

In [38]:
multi_label_accuracy(y_test, predictions)

0.376984126984127

In [39]:
multi_label_precision(y_test, predictions)

0.5858585858585859

In [40]:
multi_label_recall(y_test, predictions)

0.42822636300897166

In [41]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 72.17391304347827


In [42]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.6790890269151139
Precision for Action-Adventure: 0.625
Recall for Action-Adventure: 0.42780748663101603

Accuruacy for Romance: 0.7846790890269151
Precision for Romance: 0.5147058823529411
Recall for Romance: 0.330188679245283

Accuruacy for Horror-Thriller: 0.6521739130434783
Precision for Horror-Thriller: 0.59375
Recall for Horror-Thriller: 0.3958333333333333

Accuruacy for Comedy: 0.6190476190476191
Precision for Comedy: 0.5728643216080402
Recall for Comedy: 0.5352112676056338

Accuruacy for Science Fiction: 0.8737060041407867
Precision for Science Fiction: 0.2222222222222222
Recall for Science Fiction: 0.03571428571428571



Regular Neural Network

In [43]:
normal_nn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
normal_nn.add(e)
normal_nn.add(Flatten())
normal_nn.add(Dense(256, activation='relu'))
normal_nn.add(Dense(len(genre_dict), activation='sigmoid'))
normal_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
normal_nn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 2s - loss: 0.6169 - raw_multi_label_accuracy: 0.0818 - val_loss: 0.5998 - val_raw_multi_label_accuracy: 0.1131
Epoch 2/1000
 - 2s - loss: 0.5803 - raw_multi_label_accuracy: 0.0761 - val_loss: 0.5947 - val_raw_multi_label_accuracy: 0.1083
Epoch 3/1000
 - 1s - loss: 0.5558 - raw_multi_label_accuracy: 0.1597 - val_loss: 0.5903 - val_raw_multi_label_accuracy: 0.1494
Epoch 4/1000
 - 2s - loss: 0.5223 - raw_multi_label_accuracy: 0.2511 - val_loss: 0.5970 - val_raw_multi_label_accuracy: 0.1331
Epoch 5/1000
 - 1s - loss: 0.4640 - raw_multi_label_accuracy: 0.3733 - val_loss: 0.6027 - val_raw_multi_label_accuracy: 0.1970
Epoch 6/1000
 - 1s - loss: 0.3614 - raw_multi_label_accuracy: 0.5831 - val_loss: 0.5940 - val_raw_multi_label_accuracy: 0.1579
Epoch 7/1000
 - 1s - loss: 0.2481 - raw_multi_label_accuracy: 0.7657 - val_loss: 0.6225 - val_raw_multi_label_accuracy: 0.1867
Epoch 8/1000
 - 1s - loss: 0.1667 - raw_multi_label_accuracy: 0.

In [44]:
predictions = nn_output_to_predictions(normal_nn.predict(x_test_seq))

In [45]:
multi_label_accuracy(y_test, predictions)

0.225327812284334

In [46]:
multi_label_precision(y_test, predictions)

0.5115894039735099

In [47]:
multi_label_recall(y_test, predictions)

0.24327122153209108

In [48]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 69.27536231884058


In [49]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.6459627329192547
Precision for Action-Adventure: 0.58
Recall for Action-Adventure: 0.31016042780748665

Accuruacy for Romance: 0.7846790890269151
Precision for Romance: 0.5454545454545454
Recall for Romance: 0.11320754716981132

Accuruacy for Horror-Thriller: 0.6004140786749482
Precision for Horror-Thriller: 0.4942528735632184
Recall for Horror-Thriller: 0.22395833333333334

Accuruacy for Comedy: 0.5424430641821946
Precision for Comedy: 0.46923076923076923
Recall for Comedy: 0.2863849765258216

Accuruacy for Science Fiction: 0.8902691511387164
Precision for Science Fiction: 1.0
Recall for Science Fiction: 0.05357142857142857



LSTM

In [66]:
from keras.layers import LSTM
lstm_model = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
lstm_model.add(e)
lstm_model.add(LSTM(100, dropout=0.25, recurrent_dropout=0.25))
rnn.add(Dense(256, activation='relu'))#experiment with this here and not here
lstm_model.add(Dense(len(genre_dict), activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
lstm_model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 9s - loss: 0.6425 - raw_multi_label_accuracy: 0.0932 - val_loss: 0.5899 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 7s - loss: 0.5926 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5813 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 7s - loss: 0.5876 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5839 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 7s - loss: 0.5851 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5805 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 7s - loss: 0.5807 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5793 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 7s - loss: 0.5728 - raw_multi_label_accuracy: 0.0040 - val_loss: 0.5729 - val_raw_multi_label_accuracy: 0.0395
Epoch 7/1000
 - 7s - loss: 0.5573 - raw_multi_label_accuracy: 0.1312 - val_loss: 0.5572 - val_raw_multi_label_accuracy: 0.1286
Epoch 8/1000
 - 8s - loss: 0

In [67]:
predictions = nn_output_to_predictions(lstm_model.predict(x_test_seq))

In [68]:
multi_label_accuracy(y_test, predictions)

0.4839544513457561

In [69]:
multi_label_precision(y_test, predictions)

0.6149545772187281

In [70]:
multi_label_recall(y_test, predictions)

0.5914423740510696

In [71]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 74.24430641821947


In [72]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.722567287784679
Precision for Action-Adventure: 0.6606060606060606
Recall for Action-Adventure: 0.5828877005347594

Accuruacy for Romance: 0.7556935817805382
Precision for Romance: 0.4318181818181818
Recall for Romance: 0.3584905660377358

Accuruacy for Horror-Thriller: 0.6542443064182195
Precision for Horror-Thriller: 0.5706214689265536
Recall for Horror-Thriller: 0.5260416666666666

Accuruacy for Comedy: 0.6997929606625258
Precision for Comedy: 0.646551724137931
Recall for Comedy: 0.704225352112676

Accuruacy for Science Fiction: 0.8799171842650103
Precision for Science Fiction: 0.4666666666666667
Recall for Science Fiction: 0.25



simple rnn

In [65]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

from keras.layers import SimpleRNN
rnn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
rnn.add(e)
rnn.add(SimpleRNN(32, activation = 'relu'))
rnn.add(Dense(256, activation='relu'))
rnn.add(Dense(len(genre_dict), activation='sigmoid'))
rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
rnn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 3s - loss: 0.6983 - raw_multi_label_accuracy: 0.1373 - val_loss: 0.6678 - val_raw_multi_label_accuracy: 0.1458
Epoch 2/1000
 - 1s - loss: 0.6321 - raw_multi_label_accuracy: 0.0864 - val_loss: 0.6068 - val_raw_multi_label_accuracy: 0.0067
Epoch 3/1000
 - 1s - loss: 0.5878 - raw_multi_label_accuracy: 0.0026 - val_loss: 0.5922 - val_raw_multi_label_accuracy: 0.0035
Epoch 4/1000
 - 1s - loss: 0.5633 - raw_multi_label_accuracy: 0.0233 - val_loss: 0.5829 - val_raw_multi_label_accuracy: 0.0103
Epoch 5/1000
 - 2s - loss: 0.5372 - raw_multi_label_accuracy: 0.0615 - val_loss: 0.5804 - val_raw_multi_label_accuracy: 0.0103
Epoch 6/1000
 - 1s - loss: 0.4992 - raw_multi_label_accuracy: 0.1941 - val_loss: 0.5800 - val_raw_multi_label_accuracy: 0.0602
Epoch 7/1000
 - 2s - loss: 0.4357 - raw_multi_label_accuracy: 0.4051 - val_loss: 0.6132 - val_raw_multi_label_accuracy: 0.0712
Epoch 8/1000
 - 1s - loss: 0.3575 - raw_multi_label_accuracy: 0.

In [59]:
predictions = nn_output_to_predictions(rnn.predict(x_test_seq))

In [60]:
multi_label_accuracy(y_test, predictions)

0.28112491373360926

In [61]:
multi_label_precision(y_test, predictions)

0.4123427672955975

In [62]:
multi_label_recall(y_test, predictions)

0.3618012422360249

In [63]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 64.55486542443064


In [64]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.5548654244306418
Precision for Action-Adventure: 0.4375
Recall for Action-Adventure: 0.5240641711229946

Accuruacy for Romance: 0.7619047619047619
Precision for Romance: 0.3902439024390244
Recall for Romance: 0.1509433962264151

Accuruacy for Horror-Thriller: 0.546583850931677
Precision for Horror-Thriller: 0.4354066985645933
Recall for Horror-Thriller: 0.4739583333333333

Accuruacy for Comedy: 0.5093167701863354
Precision for Comedy: 0.4230769230769231
Recall for Comedy: 0.30985915492957744

Accuruacy for Science Fiction: 0.855072463768116
Precision for Science Fiction: 0.18181818181818182
Recall for Science Fiction: 0.07142857142857142

